In [ ]:
import pandas as pd
import datetime as dt
from datetime import date
import numpy as np
import requests
from src.fortnite_api import *
from src.azure_db import *
from dotenv import load_dotenv



In [2]:
print(os.getenv('api_fortnite_com_header_api_value'))

125153d4-87fc-4c44-8da2-db1c21c46b0d


for cil20-24 and cre95360, the search has to occur with the psn information.  
Interesting would be to store the information directly within the database as platform information for more automation

In [84]:
#connect to Azure database
def azure_db_connect():
    """Connect to Azure DB. The connection is using credential stored in created environment

    Returns:
        variable : connection engine
    """
    print('Starting connection to Azure SQL Database...')
    db_server = os.getenv('az_server')
    db_name = os.getenv('az_database')
    db_uid = os.getenv('az_username')
    db_pwd = os.getenv('az_password')

    conn_str = (
        f"Driver={{ODBC Driver 18 for SQL Server}};"
        f"Server={db_server};"
        f"Database={db_name};"
        f"Uid={db_uid};"
        f"Pwd={db_pwd};"
        "Encrypt=yes;"
        "TrustServerCertificate=no;")
    
    params = urllib.parse.quote_plus(conn_str)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

    # Test connection
    with engine.connect() as connection:
        connection.execute(text("SELECT 1"))
        
    print("Connected to Azure SQL Database")
    
    return engine
engine = azure_db_connect()



Starting connection to Azure SQL Database...
Connected to Azure SQL Database


In [11]:
def retrieve_existing_data(engine):
    """
    Retrieve existing players from the database.
    """
    players_df = pd.read_sql("SELECT * FROM fortnite_player", con=engine)
    players_stats_hist = pd.read_sql("SELECT * FROM fortnite_player_stats", con=engine)
    return players_df, players_stats_hist

players_df, players_stats_hist = retrieve_existing_data(engine)

In [88]:
api_header = os.getenv('api_fortnite_com_header_api_type')
api_value = os.getenv('api_fortnite_com_header_api_value')
header = {api_header: api_value}

In [16]:
accountId = 'seb24stutt'
platform = 'psn'

dict_epic_2_platform = dict(zip(players_df['player_id'], players_df['platform']))

url = f'https://fortnite-api.com/v2/stats/br/v2?name={accountId}&accountType={platform}'

#https://api.fortnitetracker.com/v1/powerrankings/{platform}/{region}/{epic

header = {api_header:api_value}
response_user_ID = requests.get(url, headers=header)
#Limit the response to only the account_id
response = response_user_ID.json()

test = pd.DataFrame(response['data']['stats']['all']).T

test

,score,scorePerMin,scorePerMatch,wins,top3,top5,top6,top10,top12,top25,kills,killsPerMin,killsPerMatch,deaths,kd,matches,winRate,minutesPlayed,playersOutlived,lastModified
overall,1186378,25.526,314.439,381,444,816,637,145,1262,246,8996,0.194,2.384,3392,2.652,3773,10.098,46478,266404,2025-12-22T12:42:26Z
solo,114900,27.68,293.862,15,NaN,NaN,NaN,145,NaN,246,1108,0.267,2.834,376,2.947,391,3.836,4151,27113,2025-12-22T12:42:26Z
duo,678305,25.308,300.934,175,NaN,816,NaN,NaN,1262,NaN,5014,0.187,2.224,2079,2.412,2254,7.764,26802,157426,2025-12-21T15:56:56Z
squad,385422,25.37,350.384,187,444,NaN,637,NaN,NaN,NaN,2802,0.184,2.547,913,3.069,1100,17,15192,80060,2025-12-06T21:25:42Z
ltm,265053,24.97,332.563,109,NaN,NaN,NaN,NaN,NaN,NaN,1977,0.186,2.481,688,2.874,797,13.676,10615,56427,2025-12-06T21:25:42Z


In [89]:
def get_stats(players_df):
    
    dict_epic_id = dict(zip(players_df['player_id'], players_df['platform']))
    
    player_stats = pd.DataFrame()
    #Create blank data list to store Stats inside
    data = []

    #Loop the player's stats and saved them in the list
    for key, value in dict_epic_id.items():
        url_player_stats = f'https://fortnite-api.com/v2/stats/br/v2?name={key}&accountType={value}'       
        response_player_stat = requests.get(url_player_stats, headers = header)
        #Manage potential errors in the API call
        if response_player_stat.status_code != 200:
            print(f"Error fetching data for player {key}: {response_player_stat.status_code}")
            continue
        
        time.sleep(1)  #To avoid hitting the API rate limit

        data_json_stats = response_player_stat.json()
        stats = pd.DataFrame(data_json_stats['data']['stats']['all']).T
        stats = stats[['wins','kd','winRate','top3','top5','top6','top10','top12','top25','kills','matches','minutesPlayed','score','playersOutlived','lastModified']]
        stats.columns = ['top_1', 'kill_death', 'win_rate','top_3','top_5', 'top_6','top_10', 'top_12', 'top_25', 'kills','match_played', 'min_played', 'score', 'players_outlived', 'last_modified']
        stats['player_id'] = key
        data.append(stats)

    for i in range(len(data)):
        player_stats = pd.concat([player_stats, data[i]])
        player_stats.drop(['last_modified'], axis='columns', inplace=True)

    return player_stats

player_stats = get_stats(players_df)



Error fetching data for player Thomas484848: 404


In [91]:
def clean_current_stats(players_stats_current):

    #avatar_dict = dict(zip(players_df['player_id'], players_df['avatar_url']))

    #Set the columns as int where necessary
    players_stats_current[['top_1', 'top_3', 'top_5', 'top_6', 'top_10',
        'top_12', 'top_25', 'kills', 'match_played', 'min_played', 'score',
        'players_outlived']] = players_stats_current[['top_1', 'top_3', 'top_5', 'top_6', 'top_10',
        'top_12', 'top_25', 'kills', 'match_played', 'min_played', 'score',
        'players_outlived']].fillna(0).astype(int)
    #Reorder columns
    players_stats_current = players_stats_current[['player_id', 'top_1', 'top_3', 'top_5', 'top_6', 'top_10',
        'top_12', 'top_25', 'kills', 'kill_death', 'win_rate','match_played', 'min_played', 'score',
        'players_outlived']]
    #rename column to Match Played
    players_stats_current = players_stats_current.rename(columns={'Match Player':'Match Played'})

    #Create a column containing the update's date
    today = date.today()
    players_stats_current['update_date'] = today

    #Add additional Statistics
    players_stats_current['kills_per_match'] = players_stats_current['kills']/players_stats_current['match_played']
    players_stats_current['kills_per_min'] = players_stats_current['kills']/players_stats_current['min_played']
    players_stats_current['score_per_match'] = players_stats_current['score']/players_stats_current['match_played']
    players_stats_current['score_per_min'] = players_stats_current['score']/players_stats_current['min_played']
    players_stats_current = players_stats_current.reset_index()
    players_stats_current.rename(columns={'index': 'type_party'}, inplace=True)
    players_stats_current['ID'] = players_stats_current['type_party']+'-'+players_stats_current['player_id']
    #the fortnite-api.com includes 'ltm' stats that we don't want to keep
    players_stats_current = players_stats_current.loc[players_stats_current['type_party']!='ltm']
    players_stats_current = players_stats_current.loc[players_stats_current['type_party']!='overall']

    return players_stats_current

players_stats_current = clean_current_stats(player_stats)
    

C:\Users\sebdu\AppData\Local\Temp\ipykernel_17640\428502826.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  'players_outlived']].fillna(0).astype(int)


In [95]:
players_stats_hist

,type_party,player_id,score_evo,top_1_evo,top_3_evo,top_5_evo,top_6_evo,top_10_evo,top_12_evo,top_25_evo,...,win_rate_value,match_played_value,kills_value,min_played_value,kills_per_min_value,kills_per_match_value,score_per_min_value,score_per_match_value,players_outlived_value,update_date
0,duo,Wayard12,0,0,0,0,0,0,0,0,...,0.00,3,10,39,0.256410,3.333333,26.923077,350.000000,72,2025-12-27
1,duo,cedrike,0,0,0,0,0,0,0,0,...,0.00,3,9,36,0.250000,3.000000,20.916667,251.000000,0,2025-12-27
2,solo,Thomas484848,0,0,0,0,0,0,0,0,...,0.00,1,3,18,0.166667,3.000000,24.722222,445.000000,0,2025-12-27
3,solo,Wayard12,0,0,0,0,0,0,0,0,...,0.00,1,0,17,0.000000,0.000000,20.882353,355.000000,0,2025-12-27
4,solo,cedrike,0,0,0,0,0,0,0,0,...,0.00,56,31,5,6.200000,0.553571,15.800000,1.410714,0,2025-12-27
5,solo,cre95360,0,0,0,0,0,0,0,0,...,0.00,1,0,12,0.000000,0.000000,16.833333,202.000000,0,2025-12-27
6,squad,Thomas484848,0,0,0,0,0,0,0,0,...,0.00,6,1,37,0.027027,0.166667,17.810811,109.833333,207,2025-12-27
7,squad,Wayard12,0,0,0,0,0,0,0,0,...,0.00,27,58,277,0.209386,2.148148,23.624549,242.370370,615,2025-12-27
8,squad,cedrike,0,0,0,0,0,0,0,0,...,0.00,7,44,103,0.427184,6.285714,25.291262,372.142857,133,2025-12-27
9,squad,cre95360,0,0,0,0,0,0,0,0,...,0.00,1,2,14,0.142857,2.000000,21.071429,295.000000,0,2025-12-27


In [93]:
def clean_upload_stats(players_stats_hist, players_stats_current):
    #From the historical stats, keep only the necessary columns and the latest update (not the evo stats)
    players_stats_hist_update = players_stats_hist.iloc[:, np.r_[0:2,20:len(players_stats_hist.columns)]].copy()
    players_stats_current_update = players_stats_current.copy()

    # players_stats_hist = players_stats_hist.loc[players_stats_hist['update_date'] == players_stats_hist['update_date'].max()]
    #Remove the '_value' suffix from the historical stats columns to merge properly
    players_stats_hist_update.columns = players_stats_hist_update.columns.str.replace('_value','')

    #Create an ID column to merge both dataframes
    players_stats_hist_update['ID'] = players_stats_hist_update['type_party']+'-'+players_stats_hist_update['player_id']
    #players_stats_current_update['ID'] = players_stats_current_update['index']+'-'+players_stats_current_update['player_id']

    #Define columns to calculate evolution (remove the one for which evolution is not relevant)
    stats_columns = ['score', 'top_1', 'top_3', 'top_5', 'top_6', 'top_10', 'top_12',
            'top_25', 'kill_death', 'win_rate', 'match_played', 'kills', 'min_played', 'kills_per_min', 'kills_per_match', 'score_per_match', 'score_per_min', 'players_outlived']

    #Calculate the evolution by subtracting current stats with historical stats
    players_stats_current_update = players_stats_current.set_index('ID')[stats_columns].subtract(players_stats_hist_update.set_index('ID')[stats_columns])
    #Merge the evolution stats with the current stats to have both in the same dataframe
    players_stats_current_update = players_stats_current_update.merge(players_stats_current, on='ID', suffixes=('_evo','_value'))

    #Reorder columns of the dataframe before uploading to the SQL Table
    players_columns_order = ['type_party', 'player_id', 'score_evo', 'top_1_evo', 'top_3_evo','top_5_evo', 'top_6_evo', 'top_10_evo', 'top_12_evo', 'top_25_evo',
        'kill_death_evo', 'win_rate_evo', 'match_played_evo', 'kills_evo','min_played_evo', 'kills_per_min_evo', 'kills_per_match_evo','score_per_min_evo', 
        'score_per_match_evo', 'players_outlived_evo','score_value', 'top_1_value', 'top_3_value', 'top_5_value','top_6_value', 'top_10_value', 'top_12_value', 
        'top_25_value','kill_death_value', 'win_rate_value', 'match_played_value','kills_value', 'min_played_value', 'kills_per_min_value',
        'kills_per_match_value', 'score_per_min_value', 'score_per_match_value', 'players_outlived_value', 'update_date']

    players_stats_current_update = players_stats_current_update[players_columns_order]

    return players_stats_current_update

players_stats_upload = clean_upload_stats(players_stats_hist, players_stats_current)

In [ ]:
players_stats_upload.to_sql('fortnite_player_stats_hist', con=engine, if_exists='append', index=False)

18

In [99]:
players_stats_upload.fillna(0).to_sql('fortnite_player_stats_hist', con=engine, if_exists='append', index=False)

C:\Users\sebdu\AppData\Local\Temp\ipykernel_17640\1160631891.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  players_stats_upload.fillna(0).to_sql('fortnite_player_stats_hist', con=engine, if_exists='append', index=False)


18

In [34]:
#Connect to Azure DB
print('Connecting to the database...')
engine = azure_db_connect()

#Retrieve existing players from DB
print('Retrieving existing players and history stats from the database...')
players_df, players_stats_hist = retrieve_existing_data(engine)

Connecting to the database...
Starting connection to Azure SQL Database...
Starting connection to Azure SQL Database...
Starting connection to Azure SQL Database...


RetryError: RetryError[<Future at 0x14703e74ec0 state=finished raised KeyError>]

In [6]:
def find_epic_id(player_id):
    """Find a player Epic ID according to his display name

    Args:
        player_id (str): The display name of the player.

    Returns:
        str: The Epic ID of the player if found, otherwise None.
    """
    #get the epi credentials for API Access
    header_api = {'Authorization':'125153d4-87fc-4c44-8da2-db1c21c46b0d'}

    #API address to get Epic ID from display name
    url_user_id = f'https://fortnite-api.com/v2/stats/br/v2?name={accountId}'
    #API Call to get Epic ID
    response_user_ID = requests.get(url_user_id+str(player_id), headers = header_api)
    #Limit the response to only the account_id
    epic_id = response_user_ID.json().get('account_id')
    #Manage cases where Epic ID is not found
    if epic_id is not None:
        return epic_id
    else:
        return print("Epic ID not found")

In [7]:
find_epic_id('cedrike')

Epic ID not found
